In [7]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# --- Define Data ---
demand_data = pd.read_excel('Realistic Data Demand.xlsx')
demand = demand_data['Demand'].tolist()
T = len(demand)  # 自动设置时间段数
generators = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8']

min_uptime = {'G1': 168, 'G2': 24, 'G3': 24, 'G4': 12, 'G5': 8, 'G6': 8, 'G7': 0, 'G8': 0}
min_downtime = {'G1': 168, 'G2': 12, 'G3': 12, 'G4': 8, 'G5': 4, 'G6': 4, 'G7': 0, 'G8': 0}
initial_on_time = {'G1': 85, 'G2': 0, 'G3': 20, 'G4': 15, 'G5': 6, 'G6': 5, 'G7': 0, 'G8': 0}
initial_off_time = {'G1': 0, 'G2': 10, 'G3': 0, 'G4': 0, 'G5': 0, 'G6': 0, 'G7': 3, 'G8': 12}
min_capacity = {'G1': 240, 'G2': 235, 'G3': 210, 'G4': 32, 'G5': 480, 'G6': 195, 'G7': 0, 'G8': 0}
max_capacity = {'G1': 480, 'G2': 590, 'G3': 520, 'G4': 406, 'G5': 870, 'G6': 350, 'G7': 735, 'G8': 1410}
startup_cost = {'G1': 10380, 'G2': 33590, 'G3': 0, 'G4': 23420, 'G5': 0, 'G6': 0, 'G7': 0, 'G8': 0}
no_load_cost = {'G1': 0, 'G2': 530, 'G3': 490, 'G4': 395, 'G5': 830, 'G6': 255, 'G7': 0, 'G8': 0}
variable_cost = {'G1': 7.7, 'G2': 16.3, 'G3': 17, 'G4': 23.7, 'G5': 40, 'G6': 45, 'G7': 75, 'G8': 77}

lambda_penalty = 100  #

pRE = demand_data['Renewable Scenario 2'].tolist()
alpha = 0.5

# --- Create Gurobi Model ---
model = gp.Model('Unit Commitment')  # Model name: Unit Commitment

# --- Define Decision Variables ---
# u[g, t]: Binary variable indicating whether generator g is ON (1) or OFF (0) at time t
u = model.addVars(generators, range(1, T+1), vtype=GRB.BINARY, name='u')

# v[g, t]: Binary variable indicating whether generator g is started up at time t
v = model.addVars(generators, range(1, T+1), vtype=GRB.BINARY, name='v')

# w[g, t]: Binary variable indicating whether generator g is shut down at time t
w = model.addVars(generators, range(1, T+1), vtype=GRB.BINARY, name='w')

# p[g, t]: Continuous variable representing the power output of generator g at time t
p = model.addVars(generators, range(1, T+1), vtype=GRB.CONTINUOUS, name='p')

# s[t]: Continuous variable representing curtailed energy at time t 
s = model.addVars(range(1, T+1), vtype=GRB.CONTINUOUS, name='s')

# --- Set Objective Function ---
# The objective is to minimize the total cost, which includes:
# 1. No-load cost (fixed operation cost)
# 2. Variable generation cost
# 3. Start-up cost
# 4. Cost for curtailed energy (can be replaced with a penalty factor, lambda_penalty)
model.setObjective(
    gp.quicksum(no_load_cost[g] * u[g, t] + variable_cost[g] * p[g, t] + startup_cost[g] * v[g, t]
                for g in generators for t in range(1, T+1)) +
    lambda_penalty * gp.quicksum(s[t] for t in range(1, T+1)),
    GRB.MINIMIZE
)

# --- Add Constraints ---

# 1. Power Output Constraints: Ensure power output is within min and max capacity when the unit is on
for g in generators:
    for t in range(1, T+1):
        model.addConstr(p[g, t] >= min_capacity[g] * u[g, t], name=f'min_cap_{g}_{t}')
        model.addConstr(p[g, t] <= max_capacity[g] * u[g, t], name=f'max_cap_{g}_{t}')

# 2. Demand Satisfaction
for t in range(1, T+1):
    model.addConstr(
        gp.quicksum(p[g, t] for g in generators) + alpha * pRE[t - 1] == demand[t - 1] + s[t],
        name=f'demand_{t}'
    )
    model.addConstr(s[t] >= 0, name=f's_nonneg_{t}')
    
# 3. State Transition Constraints: Define how the unit status (on/off) changes over time
for g in generators:
    for t in range(1, T+1):
        if t == 1:
            # At time 1, the previous status is determined by initial_on_time or initial_off_time
            u_prev = 1 if initial_on_time[g] > 0 else 0
        else:
            # For other time periods, u[g, t-1] determines the previous status
            u_prev = u[g, t-1]
        model.addConstr(u[g, t] == u_prev + v[g, t] - w[g, t], name=f'state_transition_{g}_{t}')

# 4. Minimum Uptime Constraints: If a unit is started up, it must stay on for at least its minimum uptime
for g in generators:
    for t in range(1, T+1):
        min_up_end = min(t + min_uptime[g] - 1, T)
        for tau in range(t+1, min_up_end+1):
            model.addConstr(v[g, t] + w[g, tau] <= 1, name=f'min_up_{g}_{t}_{tau}')

# 5. Minimum Downtime Constraints: If a unit is shut down, it must stay off for at least its minimum downtime
for g in generators:
    for t in range(1, T+1):
        min_down_end = min(t + min_downtime[g] - 1, T)
        for tau in range(t+1, min_down_end+1):
            model.addConstr(w[g, t] + v[g, tau] <= 1, name=f'min_down_{g}_{t}_{tau}')

# 6. Initial Condition Constraints: Respect minimum uptime/downtime based on initial status
for g in generators:
    if initial_on_time[g] > 0 and initial_on_time[g] < min_uptime[g]:
        # Ensure the unit stays on to fulfill the remaining minimum uptime
        remaining_uptime = min_uptime[g] - initial_on_time[g]
        for t in range(1, min(remaining_uptime + 1, T+1)):
            model.addConstr(u[g, t] == 1, name=f'initial_on_{g}_{t}')
    elif initial_off_time[g] > 0 and initial_off_time[g] < min_downtime[g]:
        # Ensure the unit stays off to fulfill the remaining minimum downtime
        remaining_downtime = min_downtime[g] - initial_off_time[g]
        for t in range(1, min(remaining_downtime + 1, T+1)):
            model.addConstr(u[g, t] == 0, name=f'initial_off_{g}_{t}')

# --- Solve the Model ---
model.optimize()

# --- Output Results ---
if model.status == GRB.OPTIMAL:
    print(f"Optimal Total Cost: {model.objVal}")
    for t in range(1, T+1):
        print(f"\nTime Period {t}:")
        for g in generators:
            print(f"  Unit {g}: Status={u[g,t].x}, Power={p[g,t].x}, Start-up={v[g,t].x}, Shutdown={w[g,t].x}")
        print(f"  Curtailed Energy: {s[t].x}")
else:
    print("No optimal solution found. Please check the model setup or data.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49480 rows, 5544 columns and 102218 nonzeros
Model fingerprint: 0x84545c12
Variable types: 1512 continuous, 4032 integer (4032 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+00, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 23932 rows and 1518 columns
Presolve time: 0.21s
Presolved: 25548 rows, 4026 columns, 62998 nonzeros
Variable types: 1296 continuous, 2730 integer (2730 binary)
Found heuristic solution: objective 1.358095e+07
Found heuristic solution: objective 1.248507e+07
Found heuristic solution: objective 1.247278e+07
Root relaxation presolved: 25548 rows, 4026 columns, 62998 nonzeros


Root relaxation: objective 8.936780e+06, 786 iterations, 0.03 seconds (0.07 work units)

    Nodes    |  